In [32]:
%pip install transformers
from transformers import pipeline

import pandas as pd

df = pd.read_csv("data/dataset.csv")

Note: you may need to restart the kernel to use updated packages.


In [34]:
labels = {
    # Valence
    "happy": [0.7, 1.0],
    "neutral_mood": [0.3, 0.7],
    "sad": [0.0, 0.3],

    # Popularity
    "high_popularity": [0.7, 1.0],
    "mid_popularity": [0.3, 0.7],
    "low_popularity": [0.0, 0.3],

    # Duration (normalized to [0.0, 1.0])
    "long_duration": [0.7, 1.0],
    "medium_duration": [0.3, 0.7],
    "short_duration": [0.0, 0.3],

    # Danceability
    "high_danceability": [0.7, 1.0],
    "mid_danceability": [0.3, 0.7],
    "low_danceability": [0.0, 0.3],

    # Energy
    "high_energy": [0.7, 1.0],
    "mid_energy": [0.3, 0.7],
    "low_energy": [0.0, 0.3],

    # Loudness (normalized to [0.0, 1.0])
    "high_loudness": [0.7, 1.0],
    "mid_loudness": [0.3, 0.7],
    "low_loudness": [0.0, 0.3],

    # Speechiness
    "high_speechiness": [0.7, 1.0],
    "mid_speechiness": [0.3, 0.7],
    "low_speechiness": [0.0, 0.3],

    # Acousticness
    "high_acousticness": [0.7, 1.0],
    "mid_acousticness": [0.3, 0.7],
    "low_acousticness": [0.0, 0.3],

    # Instrumentalness
    "high_instrumentalness": [0.7, 1.0],
    "mid_instrumentalness": [0.3, 0.7],
    "low_instrumentalness": [0.0, 0.3],

    # Liveness
    "high_liveness": [0.7, 1.0],
    "mid_liveness": [0.3, 0.7],
    "low_liveness": [0.0, 0.3],

    # Tempo (normalized to [0.0, 1.0])
    "high_tempo": [0.7, 1.0],
    "mid_tempo": [0.3, 0.7],
    "low_tempo": [0.0, 0.3],

    #Explicity
    "explicit": True,
    "not_explicit": False,
}

In [36]:
# Define min and max values for each feature (adjust based on your dataset)
feature_ranges = {
    "valence": (0.0, 1.0),
    "popularity": (0, 100),
    "duration_ms": (8586, 5237295),  # Adjust based on your dataset
    "danceability": (0.0, 0.985),
    "energy": (0.0, 1.0),
    "loudness": (-49.531, 4.532),  # Loudness ranges from -60 dB to 0 dB
    "speechiness": (0.0, 0.965),
    "acousticness": (0.0, 0.996),
    "instrumentalness": (0.0, 1.0),
    "liveness": (0.0, 1.0),
    "tempo": (0, 243.372),  # Tempo ranges from 0 BPM to 250 BPM
}

# Normalize features to [0.0, 1.0]
for feature, (min_val, max_val) in feature_ranges.items():
    df[f"{feature}_normalized"] = (df[feature] - min_val) / (max_val - min_val)

# Display the updated dataset
print(df.head())

   Unnamed: 0                track_id                 artists  \
0           0  5SuOikwiRyPMVoIQDJUgSV             Gen Hoshino   
1           1  4qPNDBW1i3p13qLCt0Ki3A            Ben Woodward   
2           2  1iJBSr7s7jYXzM8EGcbK5b  Ingrid Michaelson;ZAYN   
3           3  6lfxq3CG4xtTiEg7opyCyx            Kina Grannis   
4           4  5vjLSffimiIP26QG5WcN2K        Chord Overstreet   

                                          album_name  \
0                                             Comedy   
1                                   Ghost (Acoustic)   
2                                     To Begin Again   
3  Crazy Rich Asians (Original Motion Picture Sou...   
4                                            Hold On   

                   track_name  popularity  duration_ms  explicit  \
0                      Comedy          73       230666     False   
1            Ghost - Acoustic          55       149610     False   
2              To Begin Again          57       210826     False   


In [38]:
for index, rows in df.iterrows():
    if rows["track_genre"] not in labels:
        labels[rows["track_genre"]] = None

print(labels.keys())

dict_keys(['happy', 'neutral_mood', 'sad', 'high_popularity', 'mid_popularity', 'low_popularity', 'long_duration', 'medium_duration', 'short_duration', 'high_danceability', 'mid_danceability', 'low_danceability', 'high_energy', 'mid_energy', 'low_energy', 'high_loudness', 'mid_loudness', 'low_loudness', 'high_speechiness', 'mid_speechiness', 'low_speechiness', 'high_acousticness', 'mid_acousticness', 'low_acousticness', 'high_instrumentalness', 'mid_instrumentalness', 'low_instrumentalness', 'high_liveness', 'mid_liveness', 'low_liveness', 'high_tempo', 'mid_tempo', 'low_tempo', 'explicit', 'not_explicit', 'acoustic', 'afrobeat', 'alt-rock', 'alternative', 'ambient', 'anime', 'black-metal', 'bluegrass', 'blues', 'brazil', 'breakbeat', 'british', 'cantopop', 'chicago-house', 'children', 'chill', 'classical', 'club', 'comedy', 'country', 'dance', 'dancehall', 'death-metal', 'deep-house', 'detroit-techno', 'disco', 'disney', 'drum-and-bass', 'dub', 'dubstep', 'edm', 'electro', 'electronic

In [41]:
text = "I want a happy song"
hypothesis_template = "This text is about {}"
classes_verbalized = labels.keys()
zeroshot_classifier = pipeline("zero-shot-classification", model="MoritzLaurer/deberta-v3-large-zeroshot-v2.0")  # change the model identifier here
output = zeroshot_classifier(text, classes_verbalized, hypothesis_template=hypothesis_template, multi_label=True)
print(output)

config.json:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/870M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.26k [00:00<?, ?B/s]

spm.model:   0%|          | 0.00/2.46M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/8.66M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/23.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

Device set to use mps:0


TypeError: 'dict_keys' object is not subscriptable